# Criptografia assimétrica com Python

Esse roteiro foca em uso prático de criptografia assimétrica, um importante mecanismo de segurança para mecanismos de troca de chave como o TLS e mecanismos de autenticação em VPNs.
<br>
<br>
Para isso, utilizaremos a nossa conhecida **[cryptography](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/)**. Também seguiremos as recomendações de algoritmos e tamanho de chave privada (RSA e 2048 bits).
<br>
<br>
Primeiro vamos importar a biblioteca.

In [ ]:
!pip install cryptography

## Gerando par de chaves

Diferente da criptografia simétrica, não geramos a chave por fora e depois aplicamos no algoritmo. Utilizamos mecanismos da própria ferramenta para gerar a chave.
<br>
<br>
Primeiro devemos gerar a chave privada, usando o seguinte comando

In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization


private_key = rsa.generate_private_key(
    public_exponent=65537, # número primo grande, no caso, esse é o F4 (número de Fermat)
                           # considerado um ponto ótimo entre eficiência e segurança
    key_size=2048, # o tamanho da chave privada
)

# Essa parte é só para mostrar a chave privada de maneira mais legível.
# Perceba que o método também é utilizado para formatar arquivos que
# podem ser gerados/salvos utilizando a função open('private_key.pem', 'wb')

private_key_pem = private_key.private_bytes(
    # Formato de arquivo que vai ser convertido
    encoding=serialization.Encoding.PEM,
    
    # Formato de chaves privadas
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    
    # Podemos cifrar a chave privada com uma chave simétrica
    encryption_algorithm=serialization.NoEncryption()
)

print("\n".join('%s' %line.decode() for line in private_key_pem.splitlines()))

Para derivar a chave pública, basta executar o método `.public_key()` da chave privada.

In [ ]:
public_key = private_key.public_key()

# Essa parte é só para mostrar a chave pública de maneira mais legível
public_key_pem = public_key.public_bytes(
    # Formato de arquivo que vai ser convertido
    encoding=serialization.Encoding.PEM,
    
    # Formato de chaves públicas (os formatos da chave privada não são
    # compatíveis aqui e vice versa)
    format=serialization.PublicFormat.SubjectPublicKeyInfo,
)

print("\n".join('%s' %line.decode() for line in public_key_pem.splitlines()))

## Cifrando usando chave pública

Agora que temos nosso par de chaves podemos começar a cifrar e descifrar mensagens. Se usarmos o método `.encrypt(mesage: bytes, padding: Padding)`

In [ ]:
from cryptography.hazmat.primitives.asymmetric.padding import OAEP, MGF1
from cryptography.hazmat.primitives import hashes

message = b"dado que vai ser cifrado"

cipher_text = public_key.encrypt(
    message,
    # Optimal Asymmetric Encryption Padding
    OAEP(
        mgf=MGF1(algorithm=hashes.SHA256()), # Uma instância de um mask generation function. 
        algorithm=hashes.SHA256(), # Algoritmo de hashing utilizado pelo OAEP.
        label=None
    )
)

print(cipher_text)

## Descifrar usando chave privada

Já aprendemos a cifrar usando a chave pública, agora vamos descifrar usando a chave privada correspondente a chave pública que utilizamos.

In [ ]:
plaintext = private_key.decrypt(
    cipher_text,
    OAEP(
        mgf=MGF1(algorithm=hashes.SHA256()), # Uma instância de um mask generation function. 
        algorithm=hashes.SHA256(), # Algoritmo de hashing utilizado pelo OAEP.
        label=None
    )
)

print(plaintext)

# Atividade

Para este exercício de criptografia, você deve realizar as seguintes etapas:

- Gerar um par de chaves (pública e privada) com o tamanho de chave de 2048 bits, utilizando RSA.
- Definir uma mensagem para criptografar.
- Imprimir a mensagem criptografada utilizando a chave pública gerada anteriormente.
- Decifrar a mensagem utilizando a chave privada correspondente.
- Imprimir a mensagem original e verificar se ela corresponde à mensagem criptografada anteriormente.
- Salvar os arquivos das chaves em um local seguro para uso futuro.

Lembre-se de seguir as melhores práticas de segurança de criptografia para garantir a proteção das informações sensíveis.